### Washington State Crash Event Analysis
#### --- by 

In [20]:
import pandas as pd
import numpy as np
import regex as re

import requests
import asyncio
import json as js

import time

import os

pd.set_option('display.max_rows', 6)

#### Pre-defined functions for analysis

In [21]:
dir = os.path.abspath(os.path.dirname(os.getcwd())) + '/data/'

df_data = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0') 
df_data.zipCode = df_data.event_zipcode.astype(str)   # convert the default float type values into str

df_crash_type = pd.read_csv(dir + '/output/crash_type.csv')

df_data.head()

C:\Users\Chaconne\AppData\Local\Temp\ipykernel_30464\3526504653.py:3: DtypeWarning: Columns (22,23,24,302,304) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(dir + '/output/data_with_zipcode.csv').drop(axis=1, labels='Unnamed: 0')
C:\Users\Chaconne\AppData\Local\Temp\ipykernel_30464\3526504653.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_data.zipCode = df_data.event_zipcode.astype(str)   # convert the default float type values into str


,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,nmdistract6,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,event_zipcode
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_data.shape

(4132, 305)

In [23]:
# drop rows which do not have a zipcode

has_no_zipcode = df_data.zipCode.map(lambda v : v == 'nan')
df_data = df_data[df_data.zipCode != 'nan']
df_data.shape

(89, 305)

##### Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

In [26]:
df_data['is_resident'] = df_data.index.map(
    lambda i: df_data.event_zipcode[i] == df_data.dzip[i]
)

2414    False
2415    False
2434    False
        ...  
3883    False
4041    False
4058    False
Name: is_resident, Length: 89, dtype: object

##### Are there differences in the types of crashes and behavior factors in those crashes among “residents” versus those deemed to be not “from” the area?

In [ ]:
df_crash_type[4:10]

,info
4,4=Specifics Other
5,5=Specifics Unknown
6,6=Drive Off Road
7,7=Control/Traction Loss
8,"8=Avoid Collision With Vehicle, Pedestrian, An..."
9,9=Specifics Other


In [ ]:
crash_type_mapper = {
    # map a crash number to its actual meaning
    k: v for k,v in zip(
        df_crash_type['info'].map(
            lambda v: v.strip().split('=')[0].strip()
        ),
        df_crash_type['info'].map(
            lambda v: v.strip().split('=')[1].strip()
        ),
    )
}

##### Analysis of Behavioral Factors

In [ ]:
df_data.loc[:, 'race1': 'race5'].apply(
    lambda col : sum(col.isnull())
)

race1    1556
race2    4051
race3    4100
race4    4107
race5    4107
dtype: int64

- The following columns are thought to indicate whether an involved person conducted risky behavior in the crash event.
- - restraintmisuse: valued 1 when there was a restraint misuse
- - helmetmisuse: valued 1 when there was a helmet misuse
- - 